Initialize Repos

In [ ]:
!pip install gdown --upgrade
#!pip install overcomplete
#!pip install einops

In [ ]:
import os
os.makedirs("oracle_saes", exist_ok=True)
os.makedirs("oracle_selection_models", exist_ok=True)
os.makedirs("./domainbed/data", exist_ok=True)

In [ ]:
!curl -L -o ./domainbed/data/PACS.zip https://www.kaggle.com/api/v1/datasets/download/ma3ple/pacs-dataset

In [ ]:
!unzip ./domainbed/data/PACS.zip -d ./domainbed/data/
!mv -f ./domainbed/data/kfold ./domainbed/data/PACS

Get SAES

In [2]:
!gdown --folder https://drive.google.com/drive/folders/1kxAVmkUpL6fmlnZN6g7kPOrdyq7DxqsH -O ./oracle_selection_models/

^C


In [ ]:
from saetrainer_shell import run
run()